## Overview

1. Prompt the user for a show name, fetch matching shows from TV-Maze. 
   
2. Persist that JSON and load it into DuckDB for easy querying.  
   
3. Let the user pick a show ID, fetch its cast, massage the JSON, and render a table with headshots.  


### Install Dependencies

The line below installs the main dependencies

* duckdb - a in-memory scalable database solution
* jupysql - a framework for querying via SQL and rendering the results

In [2]:
%pip install jupysql duckdb duckdb-engine rich --quiet


Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext sql
%sql duckdb://

Connecting to 'duckdb://'

# TV Maze --> find tv-shows
 Establish Connection to Source, Download Data, parse it, save it, transfrom it to sql and query it


In [4]:
import requests
import json

query_term = input("Enter tv-show name you are looking for: ")
res = requests.get(
    f"https://api.tvmaze.com/search/shows?q={query_term}",
)

from pathlib import Path

# The retrieved data is for the moment stored in memory in the res object.
response_as_json = res.json()

#Extract only the bits we need
filtered = []
for entry in response_as_json:
    show = entry["show"]
    filtered.append({
        "id":       show["id"],
        "name":     show["name"],
        "language": show.get("language"),
        "genres":   show.get("genres", []),
        "summary": show.get("summary")
    })


path = Path("tv_maze_filtered.json")
if path.exists():
    print(f"Deleting {path}")
    path.unlink()


out_path = Path("tv_maze_filtered.json")
out_path.write_text(json.dumps(filtered, indent=2), encoding="utf-8")





Deleting tv_maze_filtered.json


3839

In [5]:

%%sql
SELECT *
FROM read_json_auto({{path.name}})

Running query in 'duckdb://'

id,name,language,genres,summary
49963,Game,English,['Drama'],Political drama written by Martin Lawrence & Snoop Dogg.
23647,Game,Chinese,[],None
43687,Squid Game,Korean,"['Drama', 'Thriller', 'Mystery']","Hundreds of cash-strapped players accept a strange invitation to compete in children's games. Inside, a tempting prize awaits — with deadly high stakes."
83042,The Game,English,['Thriller'],"The cat and mouse thriller tells the story of Huw Miller, a police detective still haunted by the one case that he failed to solve – a stalker who taunted his town, mercilessly toying with his victims before eventually killing them. As Huw attempts to settle into retirement, the case is never far from his mind - and when new neighbour, Patrick Harbottle moves in and utters the chilling phrase ‘catch you later' – the sign off the stalker used to taunt Huw during the investigation – Huw is determined he's finally got his man.What follows is a high stakes game of psychological chess between the two neighbours as Huw's world begins to crumble around him. Unable to bear the guilt of the stalker taking another victim under his nose, Huw risks everything to unearth the truth. But has he set his sights on the right man, or is his obsession pushing him ever closer to the brink?"
16694,Match Game,English,['Comedy'],Contestants attempt to match the answers of celebrities in the iconic game of fill in the missing blank.
50895,Game Changer,English,['Comedy'],"In this game show, the game changes every show! Players begin each round without knowing the rules -- and must figure them out while competing to win."
70384,Death's Game,Korean,"['Drama', 'Fantasy', 'Horror']",Death's Game will tell the story of a man who gets more than one second chance at life after facing death.
75361,Genius Game,English,[],"Genius Game is a brand-new reality game show that will see strategically minded players, chosen not just for their high intellectual prowess but also their social IQ, face the ultimate battle of survival. To win, contestants will participate in a host of never before seen games all designed to not only test their intellectual acumen but encourage them to carefully and creatively use the art of manipulation to outfox their opponents and win a cash prize."
45236,Darwin's Game,Japanese,"['Action', 'Anime', 'Science-Fiction']","An unknowing Sudou Kaname is invited to try out a new mysterious mobile app game called Darwin's Game, but later realizes that he's in for more than he's bargained for when he finds out that there's no way to quit the game."
84388,Match Game,English,['Comedy'],"Match Game features four contestants each week vying for the chance to win $25,000 as they attempt to match the answers of six celebrities in a game of fill-in-the-blank."


# TV Maze --> find show cast

In [6]:

query_term = input("enter tv-show id (next to name in the output table): ")
res = requests.get(
    f"https://api.tvmaze.com/shows/{query_term}/cast",
)

from pathlib import Path

# The retrieved data is for the moment stored in memory in the res object.
response_as_json = res.json()


def safe_get(d, *keys, default=None):
    for k in keys:
        if not isinstance(d, dict):
            return default
        d = d.get(k, default)
    return d


filtered = []
for entry in response_as_json:  # or wherever the list lives
    filtered.append({
        "name":             safe_get(entry, "person", "name"),
        "gender":           safe_get(entry, "person", "gender"),
        "birthday":         safe_get(entry, "person", "birthday"),
        "country":          safe_get(entry, "person", "country", "name"),
        "character played": safe_get(entry, "character", "name"),
        "character image":  safe_get(entry, "character", "image", "medium"),
    })




path = Path("tv_show_cast_filtered.json")
if path.exists():
    print(f"Deleting {path}")
    path.unlink()


out_path = Path("tv_show_cast_filtered.json")
out_path.write_text(json.dumps(filtered, indent=2), encoding="utf-8")





6499

In [7]:

import duckdb
df = duckdb.connect().execute(
    "SELECT * FROM read_json_auto('tv_show_cast_filtered.json')"
).df()


from IPython.display import HTML

# 1) Create a new “img” column of <img> tags
df['img'] = df['character image'].apply(
    lambda url: f'<img src="{url}" width="80">'
)

# 2) Pick the columns you want (img first, then the rest)
cols = ['img','character played','gender','birthday','country','name']

# 3) Render the HTML table with images unescaped
HTML( df[cols].to_html(escape=False, index=False) )


img,character played,gender,birthday,country,name
,Sung Gi Hun,Male,1972-12-15,"Korea, Republic of",Lee Jung-jae
,Hwang Joon Ho,Male,1991-08-05,"Korea, Republic of",Wi Ha Joon
,Jo Sang Woo,Male,1981-11-21,"Korea, Republic of",Park Hae Soo
,Kang Sae Byuk,Female,1994-06-23,"Korea, Republic of",Jung Ho Yun
,Front Man,Male,1970-07-12,"Korea, Republic of",Lee Byung Hun
,Jang Duk Soo,Male,1977-10-20,"Korea, Republic of",Heo Sung Tae
,Oh Il Nam,Male,1944-10-19,"Korea, Republic of",Oh Young Soo
,Ali Abdul,Male,1988-11-02,India,Anupam Tripathi
,Han Mi Nyeo,Female,1976-09-10,"Korea, Republic of",Kim Joo Ryung
,Byeong Ki,Male,NaT,None,Yoo Sung Joo
